In [1]:
from gensim import corpora, models, similarities
import nltk
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
import scipy.sparse

D:\Drace\Anaconda2\lib\site-packages\gensim\utils.py:855: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [8]:
import requests 
from gutenberg.cleanup import strip_headers
data = requests.get("http://www.gutenberg.org/files/2701/2701-0.txt")
moby_dick=data.text
moby = strip_headers(moby_dick).strip()

In [12]:
moby = [moby]

In [14]:
type(moby)

list

In [15]:
#testing purpose, should import stoplist from online
stoplist = set('for a of the and to in I'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in moby]

In [16]:
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
     for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1]
          for text in texts]

In [17]:
texts

[[u'moby',
  u'dick;',
  u'or,',
  u'whale.',
  u'by',
  u'contents',
  u'etymology.',
  u'extracts',
  u'(supplied',
  u'by',
  u'sub-sub-librarian).',
  u'chapter',
  u'1.',
  u'loomings.',
  u'chapter',
  u'2.',
  u'carpet-bag.',
  u'chapter',
  u'3.',
  u'spouter-inn.',
  u'chapter',
  u'4.',
  u'counterpane.',
  u'chapter',
  u'5.',
  u'breakfast.',
  u'chapter',
  u'6.',
  u'street.',
  u'chapter',
  u'7.',
  u'chapel.',
  u'chapter',
  u'8.',
  u'pulpit.',
  u'chapter',
  u'9.',
  u'sermon.',
  u'chapter',
  u'10.',
  u'bosom',
  u'friend.',
  u'chapter',
  u'11.',
  u'nightgown.',
  u'chapter',
  u'12.',
  u'biographical.',
  u'chapter',
  u'13.',
  u'wheelbarrow.',
  u'chapter',
  u'14.',
  u'nantucket.',
  u'chapter',
  u'15.',
  u'chowder.',
  u'chapter',
  u'16.',
  u'ship.',
  u'chapter',
  u'17.',
  u'ramadan.',
  u'chapter',
  u'18.',
  u'his',
  u'mark.',
  u'chapter',
  u'19.',
  u'prophet.',
  u'chapter',
  u'20.',
  u'all',
  u'astir.',
  u'chapter',
  u'21.',
  u'go

In [19]:
#saving to corpora dictionary to convert words to tokens soon
dictionary = corpora.Dictionary(texts)
dictionary.save('Moby.dict')

In [20]:
print dictionary

Dictionary(12221 unique tokens: [u'divinely', u'\u201cyes,', u'foul', u'four', u'd\u2019ye']...)


In [21]:
pride_prej_para = 'After walking two or three times along that part of the lane, she was tempted, by the pleasantness of the morning, to stop at the gates and look into the park. The five weeks which she had now passed in Kent had made a great difference in the country, and every day was adding to the verdure of the early trees. She was on the point of continuing her walk, when she caught a glimpse of a gentleman within the sort of grove which edged the park: he was moving that way; and fearful of its being Mr. Darcy, she was directly retreating. But the person who advanced was now near enough to see her, and stepping forward with eagerness, pronounced her name. She had turned away; but on hearing herself called, though in a voice which proved it to be Mr. Darcy, she moved again towards the gate. He had by that time reached it also, and, holding out a letter, which she instinctively took, said, with a look of haughty composure, "I have been walking in the grove some time in the hope of meeting you. Will you do me the honour of reading that letter?" And then, with a slight bow, turned again into the plantation, and was soon out of sight.'
#may have to remove stop words, just testing for now
new_vec = dictionary.doc2bow(pride_prej_para.lower().split())
print(new_vec)

[(46, 1), (226, 1), (581, 4), (710, 1), (755, 2), (834, 1), (845, 1), (1153, 2), (1331, 1), (1447, 2), (1462, 3), (1497, 1), (1739, 1), (1964, 1), (2108, 1), (2204, 2), (2214, 2), (2413, 1), (2455, 2), (2748, 1), (2982, 1), (3005, 1), (3006, 1), (3220, 1), (3238, 1), (3477, 1), (3516, 1), (3653, 1), (3680, 1), (3750, 7), (3800, 1), (3929, 1), (4043, 2), (4127, 1), (4138, 4), (4178, 2), (4411, 1), (4501, 1), (4532, 1), (4551, 1), (4739, 1), (4866, 1), (4912, 1), (5062, 2), (5108, 1), (5158, 1), (5193, 1), (5414, 1), (5424, 1), (5431, 1), (5543, 1), (5583, 1), (5627, 2), (5691, 8), (5970, 2), (6227, 1), (6244, 1), (6254, 1), (6434, 1), (6457, 1), (6562, 2), (6572, 1), (6587, 2), (6862, 1), (7236, 1), (7294, 1), (7522, 1), (7577, 2), (7643, 1), (7724, 1), (7865, 1), (7949, 1), (8128, 1), (8195, 1), (8276, 1), (8296, 1), (8328, 1), (8496, 1), (8609, 1), (8660, 1), (8705, 1), (8996, 1), (9412, 1), (9473, 1), (9590, 1), (9696, 1), (9712, 1), (9853, 1), (10022, 1), (10350, 1), (10480, 1), (10

In [ ]:
#it's better if we implement corprus streaming, note that for this toy example, this hasn't been implemented but here's the code:
'''
class MyCorpus(object):
    def __iter__(self):
        for line in open('mycorpus.txt'):
            # assume there's one document per line, tokens separated by whitespace
            yield dictionary.doc2bow(line.lower().split())
            
Similarly, to construct the dictionary without loading all texts into memory:

from six import iteritems
# collect statistics about all tokens
dictionary = corpora.Dictionary(line.lower().split() for line in open('mycorpus.txt'))
# remove stop words and words that appear only once
stop_ids = [dictionary.token2id[stopword] for stopword in stoplist
            if stopword in dictionary.token2id]
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(stop_ids + once_ids)  # remove stop words and words that appear only once
dictionary.compactify()  # remove gaps in id sequence after words that were removed
print(dictionary)
'''

In [35]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('Moby.mm', corpus)
corpus = corpora.MmCorpus('Moby.mm')

In [40]:
for doc in corpus:
    print(doc)

[(0, 11.0), (1, 10.0), (2, 63.0), (3, 28.0), (4, 20.0), (5, 2.0), (6, 5.0), (7, 19.0), (8, 2.0), (9, 6.0), (10, 2.0), (11, 2.0), (12, 4.0), (13, 17.0), (14, 4.0), (15, 39.0), (16, 3.0), (17, 10.0), (18, 2.0), (19, 5.0), (20, 7.0), (21, 4.0), (22, 2.0), (23, 3.0), (24, 19.0), (25, 2.0), (26, 2.0), (27, 4.0), (28, 5.0), (29, 2.0), (30, 2.0), (31, 2.0), (32, 3.0), (33, 27.0), (34, 2.0), (35, 2.0), (36, 4.0), (37, 2.0), (38, 3.0), (39, 4.0), (40, 2.0), (41, 40.0), (42, 2.0), (43, 4.0), (44, 12.0), (45, 37.0), (46, 27.0), (47, 2.0), (48, 4.0), (49, 2.0), (50, 50.0), (51, 9.0), (52, 2.0), (53, 3.0), (54, 32.0), (55, 2.0), (56, 2.0), (57, 5.0), (58, 89.0), (59, 7.0), (60, 14.0), (61, 6.0), (62, 9.0), (63, 3.0), (64, 5.0), (65, 4.0), (66, 2.0), (67, 2.0), (68, 2.0), (69, 2.0), (70, 2.0), (71, 2.0), (72, 2.0), (73, 5.0), (74, 2.0), (75, 2.0), (76, 2.0), (77, 166.0), (78, 7.0), (79, 4.0), (80, 6.0), (81, 3.0), (82, 4.0), (83, 2.0), (84, 2.0), (85, 2.0), (86, 2.0), (87, 3.0), (88, 2.0), (89, 2.0)

In [53]:
model = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

In [54]:
corpus_tfidf = model[corpus]
for doc in corpus_tfidf:
    print(doc)

[(11, 0.024699878024853596), (20, 0.0360529795804281), (43, 0.30599966297170678), (92, 0.63192165271262923)]


In [37]:
tfidf = models.TfidfModel(corpus)

In [42]:
corpus_tfidf = tfidf[corpus]
for doc in corpus_tfidf:
    print(doc)

[]


In [55]:
index = similarities.MatrixSimilarity(model[corpus])
index.save('Moby.index')

In [59]:
new_vec_lda = model[new_vec]

In [60]:
sims = index[new_vec_lda]

In [61]:
sims

array([ 0.99762231], dtype=float32)

In [ ]:
#something's wrong here